In [1]:
import pandas as pd

In [2]:
df_chart_tbl = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o31_empty_lab_mean_tbl.csv', index_col=False)
df_mean_chart = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o26_labevents_24hours_mean.csv', index_col=False)
df_data_chart = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\hosp\d_labitems.csv.gz')

In [3]:
"""
Δημιουργώ εκ νέου την στήλη concat
χωρίς κενά ανάμεσα στα στοιχεία.
Θα χρησιμοποιήσω αυτή τη στήλη σαν
index γιατί τα κενά μπορεί να
δημιουργήσουν πρόβλημα.
"""
df_chart_tbl["concat"] = (df_chart_tbl["subject_id"].astype(str)
                          + df_chart_tbl["hadm_id"].astype(str)
                          + df_chart_tbl["date"].astype(str)
                          + df_chart_tbl["time"].astype(str))

"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace('-','')
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace(':','')

In [4]:
# Ορίζω την στήλη concat ως index των γραμμών
df_chart_tbl = df_chart_tbl.set_index('concat')

In [5]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες
"""
df_mean_chart["concat"] = (df_mean_chart["subject_id"].astype(str)
                           + df_mean_chart["hadm_id"].astype(str)
                           + df_mean_chart["date"].astype(str)
                           + df_mean_chart["time"].astype(str))


"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_mean_chart['concat'] = df_mean_chart['concat'].str.replace('-','')
df_mean_chart['concat'] = df_mean_chart['concat'].str.replace(':','')

In [6]:
# Γεμίζω τον πίνακα
for row in df_mean_chart.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Lab
    df_chart_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [7]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o35_fill_lab_mean_itemid_header_tbl.csv', index=False)

In [8]:
df_data_chart = df_data_chart.set_index('itemid')

In [9]:
df_data_chart

,label,fluid,category,loinc_code
itemid,,,,
51905,,Other Body Fluid,Chemistry,NaN
51532,11-Deoxycorticosterone,Blood,Chemistry,NaN
51957,17-Hydroxycorticosteroids,Urine,Chemistry,NaN
51958,"17-Ketosteroids, Urine",Urine,Chemistry,NaN
52068,24 Hr,Blood,Hematology,NaN
...,...,...,...,...
52017,"Zinc, Urine",Urine,Chemistry,NaN
52420,ZZDUMMY,Urine,Hematology,NaN
51771,NaN,Blood,Chemistry,NaN


In [10]:
# Ενοποίηση τιμών σε κελιά
df_data_chart["header"] = ("Mean - "
                           + df_data_chart["label"].astype(str)
                           + " ("
                           + df_data_chart["fluid"].astype(str)
                           + ")")

In [11]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικατάσταση.
"""
for col in df_chart_tbl.columns:
    if ((col != 'subject_id') and
        (col != 'hadm_id') and
        (col != 'date') and
        (col != 'time')):
        temp = (df_data_chart.loc[int(col), 'header'])
        df_chart_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [12]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τις ονομασίες των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o36_fill_mean_lab_label_header_tbl.csv', index=False)